### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import random

random.seed(42)
np.random.seed(42)

# Cargar datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target


random_docs = random.sample(range(len(newsgroups_train.data)), 5)

for i, doc_idx in enumerate(random_docs):
    print(f"--- DOCUMENTO {i+1} (índice {doc_idx}) ---")
    print(f"Clase: {newsgroups_train.target_names[y_train[doc_idx]]}")
    print(f"Texto (primeros 200 chars): {newsgroups_train.data[doc_idx][:200]}...")

    cossim = cosine_similarity(X_train[doc_idx], X_train)[0]
    most_similar = np.argsort(cossim)[::-1][1:6]

    print("5 documentos más similares:")
    for j, sim_idx in enumerate(most_similar):
        sim_class = newsgroups_train.target_names[y_train[sim_idx]]
        print(f"  {j+1}. Similaridad: {cossim[sim_idx]:.4f}, Clase: {sim_class}")
    print()



--- DOCUMENTO 1 (índice 10476) ---
Clase: rec.sport.hockey
Texto (primeros 200 chars): This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an ...
5 documentos más similares:
  1. Similaridad: 0.2250, Clase: rec.sport.hockey
  2. Similaridad: 0.2174, Clase: talk.politics.mideast
  3. Similaridad: 0.2164, Clase: sci.crypt
  4. Similaridad: 0.2126, Clase: alt.atheism
  5. Similaridad: 0.2111, Clase: sci.crypt

--- DOCUMENTO 2 (índice 1824) ---
Clase: comp.sys.mac.hardware
Texto (primeros 200 chars): 

	I think this kind of comparison is pretty useless in general.  The
processor is only good when a good computer is designed around it adn the
computer is used in its designed purpose.  Comparing pro...
5 documentos más similares:
  1. Similaridad: 0.3542, Clase: comp.sys.mac.hardware
  2. Similaridad: 0.3132, Clase: comp.sys.mac.hardware
  3. Simila

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [2]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

def prototype_classifier(X_train, y_train, X_test):
    predictions = []

    for i in range(X_test.shape[0]):
        similarities = cosine_similarity(X_test[i], X_train)[0]
        most_similar_idx = np.argmax(similarities)
        predictions.append(y_train[most_similar_idx])

    return np.array(predictions)

# Clasificar usando prototipos
y_pred_prototype = prototype_classifier(X_train, y_train, X_test)
f1_prototype = f1_score(y_test, y_pred_prototype, average='macro')
print(f"F1-score: {f1_prototype:.4f}")



F1-score: 0.5050


**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [3]:

best_f1 = 0
best_config = {}

vectorizer_configs = [
    {'max_features': None, 'ngram_range': (1, 1), 'min_df': 1, 'max_df': 1.0},
    {'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.95},
    {'max_features': 20000, 'ngram_range': (1, 2), 'min_df': 2, 'max_df': 0.95},
    {'max_features': 15000, 'ngram_range': (1, 2), 'min_df': 3, 'max_df': 0.90},
    {'max_features': None, 'ngram_range': (1, 1), 'min_df': 5, 'max_df': 0.85}
]

models = [MultinomialNB(), ComplementNB()]
model_names = ['MultinomialNB', 'ComplementNB']

for vec_config in vectorizer_configs:
    vectorizer = TfidfVectorizer(**vec_config)
    X_train_vec = vectorizer.fit_transform(newsgroups_train.data)
    X_test_vec = vectorizer.transform(newsgroups_test.data)

    for model, model_name in zip(models, model_names):
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)
        f1 = f1_score(y_test, y_pred, average='macro')

        print(f"{model_name} con {vec_config}: F1 = {f1:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            best_config = {'vectorizer': vec_config, 'model': model_name, 'f1': f1}



MultinomialNB con {'max_features': None, 'ngram_range': (1, 1), 'min_df': 1, 'max_df': 1.0}: F1 = 0.5854
ComplementNB con {'max_features': None, 'ngram_range': (1, 1), 'min_df': 1, 'max_df': 1.0}: F1 = 0.6930
MultinomialNB con {'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.95}: F1 = 0.6149
ComplementNB con {'max_features': 10000, 'ngram_range': (1, 1), 'min_df': 2, 'max_df': 0.95}: F1 = 0.6665
MultinomialNB con {'max_features': 20000, 'ngram_range': (1, 2), 'min_df': 2, 'max_df': 0.95}: F1 = 0.5859
ComplementNB con {'max_features': 20000, 'ngram_range': (1, 2), 'min_df': 2, 'max_df': 0.95}: F1 = 0.6639
MultinomialNB con {'max_features': 15000, 'ngram_range': (1, 2), 'min_df': 3, 'max_df': 0.9}: F1 = 0.5848
ComplementNB con {'max_features': 15000, 'ngram_range': (1, 2), 'min_df': 3, 'max_df': 0.9}: F1 = 0.6517
MultinomialNB con {'max_features': None, 'ngram_range': (1, 1), 'min_df': 5, 'max_df': 0.85}: F1 = 0.6098
ComplementNB con {'max_features': None, 'ngram_r

**Mejor modelo**: ComplementNB, **Vectorizador**: {'max_features': None, 'ngram_range': (1, 1), 'min_df': 1, 'max_df': 1.0}. **F1-score**: 0.6930

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [4]:
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}
X_term_doc = X_train.T

target_words = ['computer', 'car', 'science', 'religion', 'medical']

print("Similaridad entre palabras:")
for word in target_words:
    if word in tfidfvect.vocabulary_:
        word_idx = tfidfvect.vocabulary_[word]

        word_similarities = cosine_similarity(X_term_doc[word_idx], X_term_doc)[0]

        most_similar_indices = np.argsort(word_similarities)[::-1][1:6]

        print(f"\nPalabra: '{word}'")
        print("5 palabras más similares:")
        for i, sim_idx in enumerate(most_similar_indices):
            similar_word = idx2word[sim_idx]
            similarity = word_similarities[sim_idx]
            print(f"  {i+1}. '{similar_word}' (similaridad: {similarity:.4f})")

print(f"F1-score clasificador por prototipos: {f1_prototype:.4f}")
print(f"Mejor F1-score Naive Bayes optimizado: {best_f1:.4f}")
print(f"Mejora: {((best_f1 - f1_prototype) / f1_prototype * 100):.2f}%")

Similaridad entre palabras:

Palabra: 'computer'
5 palabras más similares:
  1. 'decwriter' (similaridad: 0.1563)
  2. 'deluged' (similaridad: 0.1522)
  3. 'harkens' (similaridad: 0.1522)
  4. 'shopper' (similaridad: 0.1443)
  5. 'the' (similaridad: 0.1361)

Palabra: 'car'
5 palabras más similares:
  1. 'cars' (similaridad: 0.1797)
  2. 'criterium' (similaridad: 0.1770)
  3. 'civic' (similaridad: 0.1748)
  4. 'owner' (similaridad: 0.1689)
  5. 'dealer' (similaridad: 0.1681)

Palabra: 'science'
5 palabras más similares:
  1. 'cognitivists' (similaridad: 0.3948)
  2. 'behaviorists' (similaridad: 0.3948)
  3. 'scientific' (similaridad: 0.3466)
  4. 'empirical' (similaridad: 0.2810)
  5. 'sects' (similaridad: 0.2622)

Palabra: 'religion'
5 palabras más similares:
  1. 'religious' (similaridad: 0.2451)
  2. 'religions' (similaridad: 0.2116)
  3. 'categorized' (similaridad: 0.2039)
  4. 'purpsoe' (similaridad: 0.2008)
  5. 'crusades' (similaridad: 0.1987)

Palabra: 'medical'
5 palabras más s